<a href="https://colab.research.google.com/github/lucarenz1997/NLP/blob/main/Stage_3_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stage 3: Implementing an RAG System for Question Answering

Part 1: Model Training Steps
Objective: Developing and utilizing advanced embedding models to represent the content of Cleantech Media and Google Patent datasets and compare domain-specific embeddings to gain unique insights.

Output: Notebook with annotated model training steps

Data Preparation for Embeddings
Lead: Alvaro Cervan

Preprocessing Steps
The preprocessing steps have already been completed in the previous stage, which include:

Dropping duplicates
Setting data types
Dropping unnecessary columns
Tokenizing text data
Stopword Removal
Language detection
Translating non-English text to English
Lemmatization
These steps were applied to both datasets, media and patents, and the resulting data was saved in the data folder. We will now load the data and perform the following steps:

## SETUP & DATA LOADING

Installationen (einmalig):

In [1]:
!pip install transformers accelerate bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [2]:
# Import imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tqdm import tqdm
import random
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### LOAD & PREPARE DATA

In [3]:
prepdata_media= pd.read_csv("/content/drive/MyDrive/CLT/data/processed_media_data_backup.csv")
prepdata_patent = pd.read_csv("/content/drive/MyDrive/CLT/data/processed_patent_data_backup.csv")
print("Media Backup:")
prepdata_media.head(5)

Media Backup:


,Unnamed: 0,title,date,author,content,domain,url,processed_text
0,93320,"XPeng Delivered ~100,000 Vehicles In 2021",2022-01-02,Unknown,['Chinese automotive startup XPeng has shown o...,cleantechnica,https://cleantechnica.com/2022/01/02/xpeng-del...,chinese automotive startup XPeng show one dram...
1,93321,Green Hydrogen: Drop In Bucket Or Big Splash?,2022-01-02,Unknown,['Sinopec has laid plans to build the largest ...,cleantechnica,https://cleantechnica.com/2022/01/02/its-a-gre...,Sinopec lay plan build large green hydrogen pr...
2,98159,World’ s largest floating PV plant goes online...,2022-01-03,Unknown,['Huaneng Power International has switched on ...,pv-magazine,https://www.pv-magazine.com/2022/01/03/worlds-...,Huaneng Power International switch MW float pv...
3,98158,Iran wants to deploy 10 GW of renewables over ...,2022-01-03,Unknown,"['According to the Iranian authorities, there ...",pv-magazine,https://www.pv-magazine.com/2022/01/03/iran-wa...,accord iranian authority currently renewable e...
4,31128,Eastern Interconnection Power Grid Said ‘ Bein...,2022-01-03,Unknown,['Sign in to get the best natural gas news and...,naturalgasintel,https://www.naturalgasintel.com/eastern-interc...,sign get good natural gas news datum follow to...


In [4]:
prepdata_patent.head(5)

,publication_number,application_number,title,abstract,publication_date,inventor,processed_text
0,US-2022239235-A1,US-202217717397-A,Adaptable DC-AC Inverter Drive System and Oper...,Disclosed is an adaptable DC-AC inverter syste...,2022-07-28 00:00:00,[],disclose adaptable DC AC inverter system opera...
1,US-2022239251-A1,US-202217580956-A,System for providing the energy from a single ...,"In accordance with an example embodiment, a so...",2022-07-28 00:00:00,[],in accordance example embodiment solar energy ...
2,EP-4033090-A1,EP-21152924-A,Method for controlling a wind energy system,Verfahren zum Steuern einer Windenergieanlage ...,2022-07-27 00:00:00,"['Schaper, Ulf', 'von Aswege, Enno', 'Gerke Fu...",Verfahren zum steuern einer Windenergieanlage ...
3,EP-4033090-A1,EP-21152924-A,Method for controlling a wind energy system,Verfahren zum Steuern einer Windenergieanlage ...,2022-07-27 00:00:00,"['Schaper, Ulf', 'von Aswege, Enno', 'Gerke Fu...",Verfahren zum steuern einer Windenergieanlage ...
4,US-11396827-B2,US-202117606042-A,Control method for optimizing solar-to-power e...,A control method for optimizing a solar-to-pow...,2022-07-26 00:00:00,[],a control method optimize solar power efficien...


## Select 50–100 Relevant Paragraphs

In [ ]:
# Function to select long, unique paragraphs
#def get_paragraphs(df, min_words=40, max_paragraphs=50):
#    paragraphs = df["processed_text"].dropna().unique()
#    filtered = [p for p in paragraphs if len(p.split()) >= min_words]
#    return filtered[:max_paragraphs]
#
## Extract paragraphs
#media_paragraphs = get_paragraphs(prepdata_media, max_paragraphs=50)
#patent_paragraphs = get_paragraphs(prepdata_patent, max_paragraphs=50)
#
## Combine and export to CSV for manual processing
#selected_paragraphs = media_paragraphs + patent_paragraphs
#
#pd.DataFrame({'paragraph': selected_paragraphs}).to_csv(
#    "/content/drive/MyDrive/CLT/data/selected_paragraphs.csv", index=False
#)

# Load generated dataset with relevant paragraphs

In [ ]:
#selected_data= pd.read_csv("/content/drive/MyDrive/CLT/data/Stage_3/selected_paragraphs.csv")

### Load manually Generate QA Pairs in ChatGPT-4

In [5]:
evaluation_data= pd.read_csv("/content/drive/MyDrive/CLT/data/Stage_3/Classified_QA_Pairs_cat.csv")

In [6]:
evaluation_data.head(10)

,Paragraph,Question,Answer,Index,Category
0,chinese automotive startup XPeng show one dram...,How has XPeng's vehicle delivery performance e...,XPeng experienced a dramatic increase in vehic...,1,Factual Questions
1,chinese automotive startup XPeng show one dram...,What infrastructure growth has XPeng achieved ...,XPeng has rapidly expanded its infrastructure ...,2,Factual Questions
2,chinese automotive startup XPeng show one dram...,How does XPeng's growth compare to other elect...,"While XPeng is not directly compared to Tesla,...",3,Comparative Questions
3,Sinopec lay plan build large green hydrogen pr...,What is Sinopec's role in the green hydrogen s...,Sinopec is actively entering the green hydroge...,4,Factual Questions
4,Sinopec lay plan build large green hydrogen pr...,How does Sinopec plan to power its green hydro...,Sinopec's project will be powered by a newly b...,5,Strategic or Predictive Questions
5,Sinopec lay plan build large green hydrogen pr...,Why is Sinopec’s investment in green hydrogen ...,Sinopec’s investment is significant because it...,6,Analytical Questions
6,Huaneng Power International switch MW float pv...,What is significant about Huaneng Power Intern...,Huaneng Power International has completed the ...,7,Factual Questions
7,Huaneng Power International switch MW float pv...,What makes the Qinggang Photovoltaic Power Sta...,"The Qinggang Photovoltaic Power Station, launc...",8,Factual Questions
8,Huaneng Power International switch MW float pv...,How is Huaneng Power contributing to innovativ...,Huaneng Power is also developing a 2 GW solar ...,9,Factual Questions
9,accord iranian authority currently renewable e...,What steps is Iran taking to expand its renewa...,"Iran plans to add 10,000 MW of renewable energ...",10,Factual Questions
